不加入帳戶資料 僅使用最後20%進行驗證 ML大亂鬥 (時間是基於距離上一筆交易的時間差 而不是距離第一筆的時間差)

In [1]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, concat_ws, lag, col
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import (
    LogisticRegression, DecisionTreeClassifier,
    RandomForestClassifier, LinearSVC
)
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# 建立 Spark Session
spark = SparkSession.builder \
    .appName("SAML-D Classifier Comparison with Timing") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

# 讀取資料
file_path = r"C:\Users\Leon\Desktop\程式語言資料\python\TD-UF\Anti Money Laundering Transaction Data (SAML-D)\SAML-D.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

# 加入時間欄位與時間差
df = df.withColumn("datetime", unix_timestamp(concat_ws(" ", df["Date"], df["Time"]), "yyyy-MM-dd HH:mm:ss"))
window_spec = Window.orderBy("datetime")
df = df.withColumn("prev_time", lag("datetime", 1).over(window_spec))
df = df.withColumn("time_diff", col("datetime") - col("prev_time")).fillna({"time_diff": 0})

# 類別欄位與向量欄位
categorical_cols = [
    "Payment_currency", "Received_currency",
    "Sender_bank_location", "Receiver_bank_location",
    "Payment_type"
]
indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep") for c in categorical_cols]
encoders = [OneHotEncoder(inputCol=c + "_idx", outputCol=c + "_vec") for c in categorical_cols]
feature_cols = ["Amount", "time_diff"] + [c + "_vec" for c in categorical_cols]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# 時間排序切分
df_sorted = df.orderBy("datetime")
total_count = df_sorted.count()
train_count = int(total_count * 0.8)
train_data = df_sorted.limit(train_count)
test_data = df_sorted.subtract(train_data)

# 模型設定
models = {
    "Logistic Regression": LogisticRegression(labelCol="Is_laundering", featuresCol="features"),
    "Decision Tree": DecisionTreeClassifier(labelCol="Is_laundering", featuresCol="features"),
    "Random Forest": RandomForestClassifier(labelCol="Is_laundering", featuresCol="features", numTrees=100),
    "SVM (LinearSVC)": LinearSVC(labelCol="Is_laundering", featuresCol="features")
}

# 評估器
binary_eval = BinaryClassificationEvaluator(labelCol="Is_laundering", metricName="areaUnderROC")
precision_eval = MulticlassClassificationEvaluator(labelCol="Is_laundering", predictionCol="prediction", metricName="precisionByLabel")
recall_eval = MulticlassClassificationEvaluator(labelCol="Is_laundering", predictionCol="prediction", metricName="recallByLabel")
f1_eval = MulticlassClassificationEvaluator(labelCol="Is_laundering", predictionCol="prediction", metricName="f1")

# 執行模型與計時
print("\n📊 模型效能與執行時間比較（使用最後 20% 為測試集）：\n")
for name, clf in models.items():
    print(f"🔹 {name}")
    start = time.time()

    pipeline = Pipeline(stages=indexers + encoders + [assembler, clf])
    model = pipeline.fit(train_data)
    predictions = model.transform(test_data)

    elapsed = time.time() - start

    auc = binary_eval.evaluate(predictions)
    precision = precision_eval.evaluate(predictions)
    recall = recall_eval.evaluate(predictions)
    f1 = f1_eval.evaluate(predictions)

    print(f"   🕒 訓練+預測時間：{elapsed:.2f} 秒")
    print(f"   📈 AUC          ：{auc:.4f}")
    print(f"   🎯 Precision    ：{precision:.4f}")
    print(f"   🎯 Recall       ：{recall:.4f}")
    print(f"   🧮 F1 Score     ：{f1:.4f}\n")



📊 模型效能與執行時間比較（使用最後 20% 為測試集）：

🔹 Logistic Regression
   🕒 訓練+預測時間：300.88 秒
   📈 AUC          ：0.7632
   🎯 Precision    ：0.9989
   🎯 Recall       ：1.0000
   🧮 F1 Score     ：0.9983

🔹 Decision Tree
   🕒 訓練+預測時間：298.85 秒
   📈 AUC          ：0.3150
   🎯 Precision    ：0.9990
   🎯 Recall       ：1.0000
   🧮 F1 Score     ：0.9985

🔹 Random Forest
   🕒 訓練+預測時間：1082.97 秒
   📈 AUC          ：0.7420
   🎯 Precision    ：0.9989
   🎯 Recall       ：1.0000
   🧮 F1 Score     ：0.9983

🔹 SVM (LinearSVC)
   🕒 訓練+預測時間：306.73 秒
   📈 AUC          ：0.7523
   🎯 Precision    ：0.9989
   🎯 Recall       ：1.0000
   🧮 F1 Score     ：0.9983



指標	是否考慮機率	是否固定 threshold	適合資料不平衡？
Accuracy	❌ 不看機率	✅ 固定（通常為0.5）	❌ 容易誤導
AUC	✅ 看機率排序	❌ 不固定	✅ 穩定

使用AUC可以避免使用Accuracy導致的預測不是洗錢而變成99%的問題

指標	意義	與誤殺（誤判正常為洗錢）有關嗎？
AUC	整體排序能力，無論門檻	❌ 不直接告訴你誤殺率，但好的 AUC 通常會帶來好的 precision
Precision	預測為洗錢中，真的洗錢的比例	✅ 直接反映「誤殺多不多」
Recall	所有洗錢中，模型抓到幾個	✅ 越高越能發現詐欺，但可能會犧牲 precision（造成誤殺）
F1 Score	Precision + Recall 的平衡	✅ 綜合表現好壞的重要指標

等等考慮使用多模態+ML大亂鬥